# Focus stacking with image bunches

Example of focus stacking with automatic alignment and luminosity balance, and partial stacking of image bunches before the final stacking.

In [2]:
from focus_stack import *

In [3]:
job = StackJob("job", "E:/Focus stacking/2024-04-05 - Chironomidae")
job.add_action(AlignLayers("align_test", input_path="src"))
job.add_action(BalanceLayersLumi("balance", i_min=30, i_max=225, mask_size=0.5))
job.add_action(FocusStackBunch("bunches", PyramidStack(), frames=10, overlap=2, denoise=0.8))
job.add_action(FocusStack("stack", PyramidStack(), postfix='_pyr', denoise=0.8))

In [ ]:
job.run()

running align_test
41 files in folder: 'E:/Focus stacking/2024-04-05 - Chironomidae/src'


align_test:   0%|          | 0/41 [00:00<?, ?it/s]